In [ ]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import warnings as wr
wr.filterwarnings('ignore')
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import category_encoders as ce
from collections import deque

# sklearn imports
import sklearn
import json

from scipy.stats import uniform
from scipy.cluster.hierarchy import dendrogram, linkage

from sklearn import metrics
from sklearn import pipeline
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import neural_network
from sklearn import model_selection
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.compose import ColumnTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import LeavePOut
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection import RFE

from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier 

from sklearn.datasets import make_blobs
from sklearn.cluster import AgglomerativeClustering

from tqdm.auto import tqdm

random_seed = 42

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Load the JSON data
with open('/kaggle/input/arc-prize-2024/arc-agi_training_solutions.json', 'r') as file:
    solutions_data = json.load(file)
with open('/kaggle/input/arc-prize-2024/arc-agi_training_challenges.json', 'r') as file:
    challenges_data = json.load(file)

training_challenges_df = pd.read_json("/kaggle/input/arc-prize-2024/arc-agi_training_challenges.json")

In [ ]:
def combine_dicts(dict1, dict2):
    combined_dict = {}
    
    for key in dict1.keys():
        if key in dict2:
            combined_dict[key] = {
                'solution': [{'output': dict1[key]}],
                'test': dict2[key]['test'],
                'train': dict2[key]['train']
            }
    
    return combined_dict
training_dict = combine_dicts(solutions_data, challenges_data)
i=0
for key,value in enumerate(training_dict.items()):
    if i<1:
        print(f"key is {key}: value is{value}")
        i += 1
    else:break

In [ ]:

def create_dataframe(data):
    records = []
    
    for id_key, id_value in data.items():
        record = {}
        record['id'] = id_key
        record['solution'] = id_value['solution'][0]['output'][0]  # Assuming single solution
        record['test'] = id_value['test'][0]['input']  # Assuming single test input

        # Adding train inputs and outputs dynamically
        for i, train_case in enumerate(id_value['train']):
            record[f'train_input_{i+1}'] = train_case['input']
            record[f'train_output_{i+1}'] = train_case['output']
        
        records.append(record)
    
    df = pd.DataFrame(records)
    return df


transform_df = create_dataframe(training_dict)
transform_df.head(5)

## adding simple objects description columns

In [ ]:
def find_shapes_without_wrap(input_grid):
    from collections import deque

    def bfs(start, num):
        queue = deque([start])
        visited[start[0]][start[1]] = True
        shape_positions = [start]
        
        while queue:
            x, y = queue.popleft()
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nx, ny = x + dx, y + dy
                if 0 <= nx < len(input_grid) and 0 <= ny < len(input_grid[0]) and not visited[nx][ny] and input_grid[nx][ny] == num:
                    visited[nx][ny] = True
                    queue.append((nx, ny))
                    shape_positions.append((nx, ny))
                    
        return shape_positions
    
    shapes = []
    visited = [[False] * len(row) for row in input_grid]
    
    for i in range(len(input_grid)):
        for j in range(len(input_grid[0])):
            if not visited[i][j]:
                shape_num = input_grid[i][j]
                shape_positions = bfs((i, j), shape_num)
                shape_indices = [i * len(input_grid[0]) + j for i, j in shape_positions]
                shape_size = len(shape_positions)
                shapes.append([shape_indices, shape_size, shape_num])
                
    return shapes


In [ ]:

def find_shapes_with_wrap(input_grid):
    rows, cols = len(input_grid), len(input_grid[0])
    visited = [[False] * cols for _ in range(rows)]
    shapes = []
    
    def bfs_with_wrap(start, num, input_grid, visited):
        rows, cols = len(input_grid), len(input_grid[0])
        queue = deque([start])
        visited[start[0]][start[1]] = True
        shape_positions = [start]

        while queue:
            x, y = queue.popleft()
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                nx, ny = (x + dx) % rows, (y + dy) % cols  # Wrap around using modulo
                if not visited[nx][ny] and input_grid[nx][ny] == num:
                    visited[nx][ny] = True
                    queue.append((nx, ny))
                    shape_positions.append((nx, ny))

        return shape_positions

    for i in range(rows):
        for j in range(cols):
            if not visited[i][j]:
                num = input_grid[i][j]
                shape_positions = bfs_with_wrap((i, j), num, input_grid, visited)
                flat_positions = [x * cols + y for x, y in shape_positions]
                shapes.append([flat_positions, len(flat_positions), num])

    return shapes


In [ ]:
def apply_shape_functions(transform_df):
    for i in range(1, 11):
        input_col = f'train_input_{i}'
        output_col = f'train_output_{i}'

        # Check if the column exists
        if input_col in transform_df.columns and output_col in transform_df.columns:
            
            # Apply the function to specific columns only if the values are not null
            mask_input = transform_df[input_col].notna()
            mask_output = transform_df[output_col].notna()
            
            transform_df.loc[mask_input, f'shape_info_no_wrap_input_{i}'] = transform_df.loc[mask_input, input_col].apply(find_shapes_without_wrap)
            transform_df.loc[mask_output, f'shape_info_no_wrap_output_{i}'] = transform_df.loc[mask_output, output_col].apply(find_shapes_without_wrap)

            transform_df.loc[mask_input, f'shape_info_wrap_input_{i}'] = transform_df.loc[mask_input, input_col].apply(find_shapes_with_wrap)
            transform_df.loc[mask_output, f'shape_info_wrap_output_{i}'] = transform_df.loc[mask_output, output_col].apply(find_shapes_with_wrap)

    return transform_df

# Apply the function to the DataFrame
transform_df = apply_shape_functions(transform_df)

# Display the first 5 rows of the DataFrame
transform_df.head(1)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
from sklearn.cluster import AgglomerativeClustering


In [ ]:
#create a dendrogram
result = [(i, j, transform_df["train_input_1"][i][j]) for i in range(len(transform_df["train_input_1"])) for j in range(len(transform_df["train_input_1"][i]))]
data = np.array(result)

linked_data = sch.linkage(data, method='ward')

# Create a dendrogram
plt.figure()
dn = sch.dendrogram(Z)
plt.show()

# Assign clusters
from scipy.cluster.hierarchy import fcluster
max_d = 10  # set your threshold distance
clusters = fcluster(Z, max_d, criterion='distance')

print("Cluster assignments:", clusters)

# Plot the data in 3D with cluster labels
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data[:, 0], data[:, 1], data[:, 2], c=clusters, cmap='prism')  # color by cluster
ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.show()